# out_fiberID
* 기존변수 + u-g,g-r,r-i,i-z만 추가변수로 사용
* fiber_ID를 제외하고 진행해봄
* 제출결과 : 0.3793087274 
-> fiberID를 제외해서 0.02상승

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss


import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV, GridSearchCV

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# #colab용
# import os
# os.chdir('/content/drive/My Drive/Colab Notebooks/Dacon/천체관측/')

In [4]:
# path = "/content/drive/My Drive/Colab Notebooks/Dacon/천체관측/data/" #colab
path = "./data/" #local
train_df = pd.read_csv(path+'train.csv',index_col=0)
test_df = pd.read_csv(path+'test.csv',index_col=0)
sample_submission_df = pd.read_csv(path+'sample_submission.csv',index_col=0)
pd.options.display.max_columns = 30

In [5]:
train_df.shape, test_df.shape

((199991, 22), (10009, 21))

## DATA Setting

In [6]:
#민강 시도중 min= -20, max = 30
def del_outlier(data, min=0, max=60):
    up_idx_t=()
    dw_idx_t=()
    train_light = data.iloc[:,2:]
    for i in range(len(train_light.columns)):
        col = train_light.columns[i]
        up_idx_t+=tuple(data[data[col]>max].index)
        dw_idx_t+=tuple(data[data[col]<min].index)
    del_idx = set(up_idx_t+dw_idx_t)
    
    return data[~data.index.isin(del_idx)]

In [7]:
train_df = del_outlier(train_df, min=-60, max=60)

In [8]:
train_df.shape

(199805, 22)

In [9]:
column_number = {}
number_columns = {}
for i, column in enumerate(sample_submission_df.columns):
    column_number[column] = i
    number_columns[i] = column
    
    
def to_number(x, dic):
    return dic[x]

train_df['type_num'] = train_df['type'].apply(lambda x: to_number(x, column_number))


In [10]:
def add_minus_feature_all(data,test = False):
    '''
    combination으로 모든 변수들의 조합에 대해서 컬럼생성 
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 64개로 증가
    '''
    from itertools import combinations
    n = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
#         data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for combi in list(combinations(selected,2)):
            name_1st = str.split(combi[0],'_')[1]
            name_2nd = str.split(combi[1],'_')[1]
            data[mag+"_"+name_1st+"-"+name_2nd] = data[combi[0]]-data[combi[1]]
        n=count
    return data

In [11]:
def add_minus_feature(data,test = False):
    '''
    근접변수에 대해서만 컬럼생성 ex) 'fiberMag_u-g', 'fiberMag_g-r', 'fiberMag_r-i', 'fiberMag_i-z'
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 38개로 증가 
    '''
    from itertools import combinations
    n = 0
    m = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
#         data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for i in range(0,len(selected)):        
            if i>0: 
                name_1st = str.split(m,'_')[1]
                name_2nd = str.split(selected[i],'_')[1]
                data[mag+"_"+name_1st+"-"+name_2nd] = data[m]-data[selected[i]]
            m = selected[i]    
        n=count
    return data

In [12]:
train_df = add_minus_feature_all(train_df)
test_df = add_minus_feature_all(test_df,True)

In [13]:
train_df.shape,test_df.shape

((199805, 63), (10009, 61))

In [14]:
train_df.columns

Index(['type', 'fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
       'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
       'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
       'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
       'modelMag_z', 'type_num', 'psfMag_u-g', 'psfMag_u-r', 'psfMag_u-i',
       'psfMag_u-z', 'psfMag_g-r', 'psfMag_g-i', 'psfMag_g-z', 'psfMag_r-i',
       'psfMag_r-z', 'psfMag_i-z', 'fiberMag_u-g', 'fiberMag_u-r',
       'fiberMag_u-i', 'fiberMag_u-z', 'fiberMag_g-r', 'fiberMag_g-i',
       'fiberMag_g-z', 'fiberMag_r-i', 'fiberMag_r-z', 'fiberMag_i-z',
       'petroMag_u-g', 'petroMag_u-r', 'petroMag_u-i', 'petroMag_u-z',
       'petroMag_g-r', 'petroMag_g-i', 'petroMag_g-z', 'petroMag_r-i',
       'petroMag_r-z', 'petroMag_i-z', 'modelMag_u-g', 'modelMag_u-r',
       'modelMag_u-i', 'modelMag_u-z', 'modelMag_g-r', 'modelMag_g-i',
       'modelMag_g-z', 'modelMag_r-i', 'modelMag_r-z

### CV

In [58]:
X = train_df.drop(columns=['type', 'type_num','fiberID'], axis=1)
y = train_df['type_num']
test_X = test_df.drop(columns = ['fiberID'])

In [59]:
folds = 3
Fold = StratifiedKFold(n_splits=folds, shuffle=True,random_state=42)
kfold =KFold(n_splits=folds, shuffle=True,random_state=42)

In [60]:
params = {"max_depth" : (7,8,9),
         "learning_rate": (0.01,0.05, 0.1),
         "num_leaves": (127,255,511),
         "min_child_samples": (30,40,50),
         "lambda_l1": (0,3,5,7,9),
         "lambda_l2": (0,3,5,7,9)}

In [61]:
defaut_lgbmc = lgb.LGBMClassifier(n_estimators=2000,random_state=1029,objective ="multiclass",class_weight = 'balanced')

In [62]:
random_search  = RandomizedSearchCV(defaut_lgbmc,param_distributions=params, cv=5, verbose =100 ,n_iter = 10)

In [ ]:
#와 3시간.. fold = 3,cv =5, n_iter =2 
for fold, (train_ids, val_ids) in enumerate(Fold.split(X,y)):
    print("★", fold, "start")
    eval_set = [(X.iloc[train_ids], y.iloc[train_ids]),(X.iloc[val_ids], y.iloc[val_ids])]
    eval_names = ['train','val']
    random_search.fit(X, y, eval_set = eval_set, eval_names = eval_names, early_stopping_rounds = 100,verbose = 100)

0 start
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] num_leaves=127, min_child_samples=40, max_depth=7, learning_rate=0.05, lambda_l2=3, lambda_l1=9 
Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.485788	val's multi_logloss: 0.490045
[200]	train's multi_logloss: 0.416739	val's multi_logloss: 0.42083
[300]	train's multi_logloss: 0.384112	val's multi_logloss: 0.388309
[400]	train's multi_logloss: 0.361398	val's multi_logloss: 0.365617
[500]	train's multi_logloss: 0.344106	val's multi_logloss: 0.348333
[600]	train's multi_logloss: 0.329958	val's multi_logloss: 0.334375
[700]	train's multi_logloss: 0.318895	val's multi_logloss: 0.323518
[800]	train's multi_logloss: 0.309794	val's multi_logloss: 0.314552
[900]	train's multi_logloss: 0.302154	val's multi_logloss: 0.306934
[1000]	train's multi_logloss: 0.295802	val's multi_logloss: 0.300624
[1100]

[800]	train's multi_logloss: 0.310099	val's multi_logloss: 0.312429
[900]	train's multi_logloss: 0.302287	val's multi_logloss: 0.304552
[1000]	train's multi_logloss: 0.295991	val's multi_logloss: 0.298211
[1100]	train's multi_logloss: 0.290387	val's multi_logloss: 0.292543
[1200]	train's multi_logloss: 0.285757	val's multi_logloss: 0.287865
[1300]	train's multi_logloss: 0.281664	val's multi_logloss: 0.283731
[1400]	train's multi_logloss: 0.278105	val's multi_logloss: 0.280147
[1500]	train's multi_logloss: 0.27489	val's multi_logloss: 0.276829
[1600]	train's multi_logloss: 0.272201	val's multi_logloss: 0.274083
[1700]	train's multi_logloss: 0.270154	val's multi_logloss: 0.272042
[1800]	train's multi_logloss: 0.268204	val's multi_logloss: 0.270078
[1900]	train's multi_logloss: 0.266509	val's multi_logloss: 0.268392
[2000]	train's multi_logloss: 0.265098	val's multi_logloss: 0.266955
Did not meet early stopping. Best iteration is:
[2000]	train's multi_logloss: 0.265098	val's multi_logloss

[1900]	train's multi_logloss: 0.217492	val's multi_logloss: 0.220086
[2000]	train's multi_logloss: 0.216402	val's multi_logloss: 0.218991
Did not meet early stopping. Best iteration is:
[2000]	train's multi_logloss: 0.216402	val's multi_logloss: 0.218991
[CV]  num_leaves=511, min_child_samples=40, max_depth=9, learning_rate=0.05, lambda_l2=0, lambda_l1=7, score=0.864, total=24.4min
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 218.4min remaining:    0.0s
[CV] num_leaves=511, min_child_samples=40, max_depth=9, learning_rate=0.05, lambda_l2=0, lambda_l1=7 
Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.452735	val's multi_logloss: 0.456206
[200]	train's multi_logloss: 0.374315	val's multi_logloss: 0.376992
[300]	train's multi_logloss: 0.333844	val's multi_logloss: 0.336202
[400]	train's multi_logloss: 0.306062	val's multi_logloss: 0.308178
[500]	train's multi_logloss: 0.286458	val's multi_logloss: 0.288552
[600]	train's multi_logloss: 0

[300]	train's multi_logloss: 0.319519	val's multi_logloss: 0.323407
[400]	train's multi_logloss: 0.283141	val's multi_logloss: 0.286933
[500]	train's multi_logloss: 0.255919	val's multi_logloss: 0.25954
[600]	train's multi_logloss: 0.235663	val's multi_logloss: 0.238877
[700]	train's multi_logloss: 0.220097	val's multi_logloss: 0.223221
[800]	train's multi_logloss: 0.207731	val's multi_logloss: 0.210767
[900]	train's multi_logloss: 0.19726	val's multi_logloss: 0.200125
[1000]	train's multi_logloss: 0.188908	val's multi_logloss: 0.191567
[1100]	train's multi_logloss: 0.181933	val's multi_logloss: 0.184475
[1200]	train's multi_logloss: 0.176333	val's multi_logloss: 0.178801
[1300]	train's multi_logloss: 0.171302	val's multi_logloss: 0.173694
[1400]	train's multi_logloss: 0.167221	val's multi_logloss: 0.169499
[1500]	train's multi_logloss: 0.163736	val's multi_logloss: 0.16595
[1600]	train's multi_logloss: 0.160844	val's multi_logloss: 0.163012
[1700]	train's multi_logloss: 0.158038	val's

[1500]	train's multi_logloss: 0.361681	val's multi_logloss: 0.363772
[1600]	train's multi_logloss: 0.35488	val's multi_logloss: 0.356912
[1700]	train's multi_logloss: 0.348513	val's multi_logloss: 0.350478
[1800]	train's multi_logloss: 0.342372	val's multi_logloss: 0.344239
[1900]	train's multi_logloss: 0.336237	val's multi_logloss: 0.338154
[2000]	train's multi_logloss: 0.330512	val's multi_logloss: 0.33233
Did not meet early stopping. Best iteration is:
[2000]	train's multi_logloss: 0.330512	val's multi_logloss: 0.33233
[CV]  num_leaves=127, min_child_samples=30, max_depth=7, learning_rate=0.01, lambda_l2=7, lambda_l1=3, score=0.854, total=17.3min
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 371.7min remaining:    0.0s
[CV] num_leaves=127, min_child_samples=30, max_depth=7, learning_rate=0.01, lambda_l2=7, lambda_l1=3 
Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 1.14032	val's multi_logloss: 1.14386
[200]	train's multi_logloss: 0.

Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.442856	val's multi_logloss: 0.44584
[200]	train's multi_logloss: 0.364787	val's multi_logloss: 0.366628
[300]	train's multi_logloss: 0.319887	val's multi_logloss: 0.321319
[400]	train's multi_logloss: 0.284003	val's multi_logloss: 0.284958
[500]	train's multi_logloss: 0.256317	val's multi_logloss: 0.25721
[600]	train's multi_logloss: 0.234781	val's multi_logloss: 0.235515
[700]	train's multi_logloss: 0.21729	val's multi_logloss: 0.218075
[800]	train's multi_logloss: 0.202127	val's multi_logloss: 0.202905
[900]	train's multi_logloss: 0.18918	val's multi_logloss: 0.190031
[1000]	train's multi_logloss: 0.178771	val's multi_logloss: 0.179701
[1100]	train's multi_logloss: 0.169441	val's multi_logloss: 0.170315
[1200]	train's multi_logloss: 0.161751	val's multi_logloss: 0.162536
[1300]	train's multi_logloss: 0.155218	val's multi_logloss: 0.156006
[1400]	train's multi_logloss: 0.14863	val's multi_logl

[1200]	train's multi_logloss: 0.294411	val's multi_logloss: 0.298247
[1300]	train's multi_logloss: 0.284194	val's multi_logloss: 0.288071
[1400]	train's multi_logloss: 0.274354	val's multi_logloss: 0.278156
[1500]	train's multi_logloss: 0.264542	val's multi_logloss: 0.268366
[1600]	train's multi_logloss: 0.255068	val's multi_logloss: 0.258942
[1700]	train's multi_logloss: 0.246606	val's multi_logloss: 0.250556
[1800]	train's multi_logloss: 0.238557	val's multi_logloss: 0.242443
[1900]	train's multi_logloss: 0.23076	val's multi_logloss: 0.23459
[2000]	train's multi_logloss: 0.223294	val's multi_logloss: 0.227047
Did not meet early stopping. Best iteration is:
[2000]	train's multi_logloss: 0.223294	val's multi_logloss: 0.227047
[CV]  num_leaves=255, min_child_samples=50, max_depth=9, learning_rate=0.01, lambda_l2=0, lambda_l1=0, score=0.864, total=35.3min
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 594.3min remaining:    0.0s
[CV] num_leaves=255, min_child_samples=50, max_depth=

Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 1.08905	val's multi_logloss: 1.09263
[200]	train's multi_logloss: 0.682529	val's multi_logloss: 0.686569
[300]	train's multi_logloss: 0.52327	val's multi_logloss: 0.527239
[400]	train's multi_logloss: 0.451376	val's multi_logloss: 0.455202
[500]	train's multi_logloss: 0.412167	val's multi_logloss: 0.415465
[600]	train's multi_logloss: 0.38695	val's multi_logloss: 0.389708
[700]	train's multi_logloss: 0.369663	val's multi_logloss: 0.372255
[800]	train's multi_logloss: 0.355867	val's multi_logloss: 0.35848
[900]	train's multi_logloss: 0.343303	val's multi_logloss: 0.34597
[1000]	train's multi_logloss: 0.331077	val's multi_logloss: 0.333685
[1100]	train's multi_logloss: 0.319778	val's multi_logloss: 0.322427
[1200]	train's multi_logloss: 0.309616	val's multi_logloss: 0.312244
[1300]	train's multi_logloss: 0.300383	val's multi_logloss: 0.303172
[1400]	train's multi_logloss: 0.291664	val's multi_loglo

In [ ]:
params2 = random_search.best_params_

In [ ]:
params2

In [ ]:
random_search.best_scores_

In [ ]:
gmodel = random_search.best_estimator_

In [ ]:
gmodel

In [ ]:
gmodel.fit(X,y)

In [ ]:
y_pred = gmodel.predict_proba(test_X)

In [ ]:
y_pred

In [ ]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission_df.columns, index=sample_submission_df.index)
submission.to_csv('./submisson/submission.csv', index=True)

## Modeling

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=[15, 4])

# Plot the log loss during training
axs[0].plot(evaluation_results['Train']['multi_logloss'], label='Train')
axs[0].plot(evaluation_results['Test']['multi_logloss'], label='Test')
axs[0].set_ylabel('Log loss')
axs[0].set_xlabel('Boosting round')
axs[0].set_title('Training performance')
axs[0].legend()

# Plot feature importance
importances = pd.DataFrame({'features': lgmb_model.feature_name(), 
                            'importance': lgmb_model.feature_importance()}).sort_values('importance', ascending=False).head(10)
axs[1].bar(x=np.arange(len(importances)), height=importances['importance'])
axs[1].set_xticks(np.arange(len(importances)))
axs[1].set_xticklabels(importances['features'])
axs[1].set_ylabel('Feature importance (# times used to split)')
axs[1].set_title('Feature importance')
plt.xticks(rotation=45)

plt.show()

In [ ]:
train_y_pred = lgmb_model.predict(X_test)
real_y_pred = lgmb_model.predict(test_X)

In [ ]:
train_y_pred.shape

In [ ]:
#log_loss
log_loss(y_test,train_y_pred,labels= list(set(y_test)))

In [ ]:
lgmb_model.feature_importance()

### feature_importance

In [ ]:
pi_df = pd.DataFrame(lgmb_model.feature_importance(), index= train_X.columns,columns= ['importance']).sort_values('importance',ascending = False)
pi_df.head(10)

### 예측결과에 대해
* 12번이 모두 6번으로 몰림
중간예측 : 0(STAR_WHITE_DWARF-2160),2(STAR_BROWN_DWARF-500),7(SERENDIPITY_DISTANT-4654),11(ROSAT_D-6580)
망한예측 : 12(STAR_PN-13),17(SERENDIPITY_MANUAL-61)

In [ ]:
cm_result = pd.DataFrame(confusion_matrix(y_test,np.argmax(train_y_pred,axis = 1)))

In [ ]:
number_columns

In [ ]:
train_df.type.value_counts()

In [ ]:
cm_result

In [ ]:
plt.figure(figsize =(15,15))
sns.heatmap(cm_result/np.sum(cm_result),annot = True, cmap ='RdYlGn',fmt='.1%')
plt.ylabel("real",fontsize=20)
plt.xlabel("predict",fontsize=20)
plt.show()

# REAL

In [ ]:
train_ds = lgb.Dataset(train_X, label = train_y)

In [ ]:
lgmb_model = lgb.train(params, train_ds,num_boost_round= 2000, verbose_eval=100)

In [ ]:
y_pred = lgmb_model.predict(test_X)

In [ ]:
y_pred

In [51]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission_df.columns, index=sample_submission_df.index)
submission.to_csv('./submisson/submission.csv', index=True)